In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from importlib import reload
from matplotlib import rc
from neurobiases import (TriangularModel,
                         EMSolver,
                         TCSolver,
                         solver_utils,
                         plot)

%matplotlib inline

In [ ]:
tm = TriangularModel(
    model='linear',
    parameter_design='direct_response',
    M=10, N=20, K=2, corr_cluster=0.4, corr_back=0.1,
    coupling_distribution='gaussian',
    coupling_sparsity=0.5,
    coupling_loc=0.0,
    coupling_scale=1,
    coupling_sum=None,
    coupling_random_state=10222020,
    tuning_distribution='gaussian',
    tuning_loc=0,
    tuning_scale=1,
    tuning_sparsity=0.6,
    tuning_random_state=10222020
)
tm.plot_tuning_curves(neuron='non-target')
plt.show()

In [ ]:
tm.L.shape

In [ ]:
tm.K = 1
tm.generate_noise_structure()

In [ ]:
tm.L.shape

In [ ]:
X, Y, y = tm.generate_samples(n_samples=1000, random_state=2332)

In [ ]:
solver = TCSolver(X, Y, y, c_tuning=1e0, c_coupling=1e0, solver='cd')

In [ ]:
solver.fit_lasso()

In [ ]:
solver.a

In [ ]:
solver = EMSolver(
    X, Y, y, K=1,
    solver='ow_lbfgs',
    max_iter=20,
    tol=0,
    c_coupling=1e-4,
    c_tuning=1e-4,
    initialization='fits'
)

In [ ]:
solver_mask = EMSolver(
    X, Y, y, K=1,
    solver='ow_lbfgs',
    max_iter=20,
    tol=0,
    c_coupling=1e-4,
    c_tuning=1e-4,
    initialization='fits',
    a_mask=tm.a.ravel() != 0,
    b_mask=tm.b.ravel() != 0,
    B_mask=tm.B != 0
)

In [ ]:
plt.plot(tm.b.ravel(), color='k', linewidth=3)
plt.plot(solver.b.ravel(), color='r', linewidth=3)
plt.plot(solver_mask.b.ravel(), color='gray', linewidth=3)

In [ ]:
plt.plot(tm.a.ravel(), color='k', linewidth=3)
plt.plot(solver.a.ravel(), color='r', linewidth=3)
plt.plot(solver_mask.a.ravel(), color='gray', linewidth=3)

In [ ]:
plt.scatter(solver.Psi_tr_to_Psi(), tm.Psi)
plt.plot([700, 1200], [700, 1200])

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import FactorAnalysis

In [ ]:
fitter = LinearRegression(fit_intercept=False)

In [ ]:
fitter.fit(Y, y.ravel()).coef_

In [ ]:
solver.Psi_to_Psi_tr(tm.Psi)

In [ ]:
solver.Psi_tr

In [ ]:
plt.plot(solver.B[:, 0])

In [ ]:
Z = np.concatenate((X, Y), axis=1)
fitter.fit(Z, y.ravel())

In [ ]:
fitter.coef_

In [ ]:
residuals = y - fitter.predict(Z)

In [ ]:
fa = FactorAnalysis(n_components=1)

In [ ]:
fa.fit(np.concatenate((y, Y), axis=1))

In [ ]:
fa.components_

In [ ]:
solver.L.shape

In [ ]:
solver_mask.fit_em(verbose=True)

In [ ]:
plt.plot(tm.a.ravel(), color='k', linewidth=3)
plt.plot(solver_mask.a.ravel(), color='gray', linewidth=3)

In [ ]:
plt.plot(tm.b.ravel(), color='k', linewidth=3)
plt.plot(solver_mask.b.ravel(), color='gray', linewidth=3)